## Tree Based Ensemble Models for Price Returns Forecasting

### Approach
1. Set Up
2. Identifying Suitable Lags for Price, Volume & Uncertainty Indices
3. Modelling w/ Grid Search & Forecast Evaluation
   1. LightGBM (h = 1, 4, 12) [MOVE TO SEPARATE NOTEBOOK]
      1. Model A (With Price Returns, Price & Volume)
      2. Model B (With A + Lucey Original Price) 
      3. Model C (With A + Lucey Reddit Price)
      4. Model D (With A + LDA Reddit Price)
      5. Model E (With A + Top2Vec Reddit Price)
      6. Model F (With A + VCRIX)
      7. Policy Basd Models?

References:
* https://neptune.ai/blog/lightgbm-parameters-guide

TBD:
1. Use Technical Indicators Common Factors as additional Baseline
2. Add in More Horizon Values
3. Use AIC / BIC 
4. Add in Historical Forecast Plots

### Set Up

In [1]:
# NB config
%load_ext autoreload
%autoreload 2

# Load Libraries
import os

os.chdir("../../")
from typing import Dict, Optional
import pandas as pd
import numpy as np
from pprint import pprint
from pathlib import Path
from datetime import datetime
from darts import TimeSeries
from darts.metrics import (
    mape,
    mse,
)
from tqdm import tqdm
from darts import concatenate
from darts.models.forecasting.gradient_boosted_model import (
    LightGBMModel,
)
from sklearn.model_selection import ParameterGrid
import warnings

warnings.filterwarnings("ignore")

### Data Preparation

In [2]:
# Data Dir
data_dir = Path("forecasting/data/modelling")

# BTC-USD data
btc_usd_fp = data_dir / "btc_usd_weekly.csv"
btc_usd_df = pd.read_csv(btc_usd_fp)

# UCRY Indices data
ucry_fp = data_dir / "ucry_indices_weekly.csv"
ucry_df = pd.read_csv(ucry_fp)

#### Create ***h***-weeks Log Price Returns Time Series

In [3]:
# h = 1 (Weekly Price Returns)
btc_usd_df["Price Returns (h=1)"] = np.log1p(btc_usd_df[["Price"]].pct_change(1))

# h = 4 (4 Week Price Returns)
btc_usd_df["Price Returns (h=4)"] = np.log1p(btc_usd_df[["Price"]].pct_change(4))

# h = 12 (12 Week Price Returns)
btc_usd_df["Price Returns (h=12)"] = np.log1p(btc_usd_df[["Price"]].pct_change(12))

# Create TimeSeries
# h = 1 (Weekly Price Returns)
btc_usd1_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=1)"]].dropna(), time_col="Date"
)

# h = 4 (4 Week Price Returns)
btc_usd4_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=4)"]].dropna(), time_col="Date"
)

# h = 12 (12 Week Price Returns)
btc_usd12_ts = TimeSeries.from_dataframe(
    btc_usd_df[["Date", "Price Returns (h=12)"]].dropna(), time_col="Date"
)

### Create Price and Volume Time Series

In [4]:
price_ts = TimeSeries.from_dataframe(btc_usd_df[["Date", "Price"]], time_col="Date")

In [5]:
vol_ts = TimeSeries.from_dataframe(btc_usd_df[["Date", "Volume"]], time_col="Date")

#### Create UCRY Indices Time Series

In [6]:
# Create TimeSeries
sel_cols = ["Date", "Index Value"]
time_col = "Date"

# Lucey Price
lucey_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "Lucey-Original-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# Lucey Reddit Price
lucey_reddit_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "Lucey-Reddit-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# LDA Price
lda_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "LDA-Reddit-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# Top2Vec Price
t2v_price = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "Top2Vec-Reddit-Price"].reset_index()[sel_cols],
    time_col=time_col,
)

# VCRIX
vcrix = TimeSeries.from_dataframe(
    ucry_df[ucry_df.Index == "VCRIX"].reset_index()[sel_cols], time_col=time_col
)

#### Train Test Split Date

In [7]:
# Split into Train and Test
split_date = pd.Timestamp("20190527")

### Identify Suitable Lags for UCRY Index Time Series
* STATUS: Use **t** variables to predict **t + h** variables for now

### Random Forest Forecasting Model & Evaluation

#### Define Params Grid for Grid Search

In [8]:
# Params Grid

gbm_params_grid = {
    "boosting_type": ["gbdt", "dart"],
    "num_boost_round": [300],
    "early_stopping_rounds": [30],
    "num_iterations": [500],
    "learning_rate": [0.01, 0.1],
    "max_depth": [2, 5],
    "num_leaves": [4, 32],
    "feature_fraction": [0.8],
    "subsample": [0.2],
    "random_state": [42],
}


gbm_params_list = list(ParameterGrid(gbm_params_grid))
len(gbm_params_list)

16

### LightGBM GridSearch Helper

In [9]:
# Homemade LightGBM Grid Search


def gridsearch_GBM(
    series: TimeSeries,
    past_covariates: TimeSeries,
    forecast_horizon: int,
    lags: int,
    lags_past_covariates: int,
    verbose: bool = False,
):

    min_error = np.float("inf")
    best_params = None

    for params in tqdm(gbm_params_list):
        model = LightGBMModel(lags=lags, lags_past_covariates=lags_past_covariates)
        error = model.backtest(
            series=series,
            past_covariates=past_covariates,
            forecast_horizon=forecast_horizon,
            stride=1,
            start=split_date,
            metric=mape,
            reduction=np.mean,
            verbose=verbose,
        )
        if error < min_error:
            min_error = error
            best_params = params

    print("Average error (min_error) over all historical forecasts: %.2f" % min_error)
    print("Best Params: %s" % str(best_params))

    return best_params, min_error

#### Model A (Price Returns, Price & Volume)

##### h = 1

In [10]:
# Model A (h = 1) Past Covariates
gbmA_1_past_covs = concatenate(
    [
        btc_usd1_ts,
        price_ts.slice_intersect(btc_usd1_ts),
        vol_ts.slice_intersect(btc_usd1_ts),
    ],
    axis=1,
)

In [11]:
gbmA_1_best_params, gbmA_1_min_error = gridsearch_GBM(
    btc_usd1_ts, gbmA_1_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [12:39<00:00, 47.46s/it]

Average error (min_error) over all historical forecasts: 237.09
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 4

In [12]:
# Model A (h = 4) Past Covariates
gbmA_4_past_covs = concatenate(
    [
        btc_usd4_ts,
        price_ts.slice_intersect(btc_usd4_ts),
        vol_ts.slice_intersect(btc_usd4_ts),
    ],
    axis=1,
)

In [13]:
gbmA_4_best_params, gbmA_4_min_error = gridsearch_GBM(
    btc_usd4_ts, gbmA_4_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:17<00:00, 34.85s/it]

Average error (min_error) over all historical forecasts: 1223.52
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 12

In [14]:
# Model A (h = 12) Past Covariates
gbmA_12_past_covs = concatenate(
    [
        btc_usd12_ts,
        price_ts.slice_intersect(btc_usd12_ts),
        vol_ts.slice_intersect(btc_usd12_ts),
    ],
    axis=1,
)

In [15]:
gbmA_12_best_params, gbmA_12_min_error = gridsearch_GBM(
    btc_usd12_ts, gbmA_12_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [08:53<00:00, 33.33s/it]

Average error (min_error) over all historical forecasts: 87.78
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### Model B (Price Returns, Price & Volume + Lucey Price Index)

#### h = 1

In [16]:
# Model B (h = 1) Past Covariates
gbmB_1_past_covs = concatenate(
    [
        btc_usd1_ts,
        price_ts.slice_intersect(btc_usd1_ts),
        vol_ts.slice_intersect(btc_usd1_ts),
        lucey_price.slice_intersect(btc_usd1_ts),
    ],
    axis=1,
)

In [17]:
gbmB_1_best_params, gbmB_1_min_error = gridsearch_GBM(
    btc_usd1_ts, gbmB_1_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:26<00:00, 35.44s/it]

Average error (min_error) over all historical forecasts: 245.44
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 4

In [18]:
# Model B (h = 4) Past Covariates
gbmB_4_past_covs = concatenate(
    [
        btc_usd4_ts,
        price_ts.slice_intersect(btc_usd4_ts),
        vol_ts.slice_intersect(btc_usd4_ts),
        lucey_price.slice_intersect(btc_usd4_ts),
    ],
    axis=1,
)

In [19]:
gbmB_4_best_params, gbmB_4_min_error = gridsearch_GBM(
    btc_usd4_ts, gbmB_4_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:13<00:00, 34.59s/it]

Average error (min_error) over all historical forecasts: 995.14
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 12

In [20]:
# Model B (h = 12) Past Covariates
gbmB_12_past_covs = concatenate(
    [
        btc_usd12_ts,
        price_ts.slice_intersect(btc_usd12_ts),
        vol_ts.slice_intersect(btc_usd12_ts),
        lucey_price.slice_intersect(btc_usd12_ts),
    ],
    axis=1,
)

In [21]:
gbmB_12_best_params, gbmB_12_min_error = gridsearch_GBM(
    btc_usd12_ts, gbmB_12_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [08:56<00:00, 33.55s/it]

Average error (min_error) over all historical forecasts: 90.91
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### Model C (Price Returns, Price & Volume + Lucey Reddit Price Index)

#### h = 1

In [22]:
# Model C (h = 1) Past Covariates
gbmC_1_past_covs = concatenate(
    [
        btc_usd1_ts,
        price_ts.slice_intersect(btc_usd1_ts),
        vol_ts.slice_intersect(btc_usd1_ts),
        lucey_reddit_price.slice_intersect(btc_usd1_ts),
    ],
    axis=1,
)

In [23]:
gbmC_1_best_params, gbmC_1_min_error = gridsearch_GBM(
    btc_usd1_ts, gbmC_1_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:25<00:00, 35.35s/it]

Average error (min_error) over all historical forecasts: 235.32
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 4

In [24]:
# Model C (h = 4) Past Covariates
gbmC_4_past_covs = concatenate(
    [
        btc_usd4_ts,
        price_ts.slice_intersect(btc_usd4_ts),
        vol_ts.slice_intersect(btc_usd4_ts),
        lucey_reddit_price.slice_intersect(btc_usd4_ts),
    ],
    axis=1,
)

In [25]:
gbmC_4_best_params, gbmC_4_min_error = gridsearch_GBM(
    btc_usd4_ts, gbmC_4_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:55<00:00, 37.24s/it]

Average error (min_error) over all historical forecasts: 1531.62
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 12

In [26]:
# Model C (h = 12) Past Covariates
gbmC_12_past_covs = concatenate(
    [
        btc_usd12_ts,
        price_ts.slice_intersect(btc_usd12_ts),
        vol_ts.slice_intersect(btc_usd12_ts),
        lucey_reddit_price.slice_intersect(btc_usd12_ts),
    ],
    axis=1,
)

In [27]:
gbmC_12_best_params, gbmC_12_min_error = gridsearch_GBM(
    btc_usd12_ts, gbmC_12_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [08:54<00:00, 33.42s/it]

Average error (min_error) over all historical forecasts: 96.24
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### Model D (Price Returns, Price & Volume + LDA Reddit Price Index)

#### h = 1

In [28]:
# Model D (h = 1) Past Covariates
gbmD_1_past_covs = concatenate(
    [
        btc_usd1_ts,
        price_ts.slice_intersect(btc_usd1_ts),
        vol_ts.slice_intersect(btc_usd1_ts),
        lda_price.slice_intersect(btc_usd1_ts),
    ],
    axis=1,
)

In [29]:
gbmD_1_best_params, gbmD_1_min_error = gridsearch_GBM(
    btc_usd1_ts, gbmD_1_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:28<00:00, 35.56s/it]

Average error (min_error) over all historical forecasts: 281.33
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 4

In [30]:
# Model D (h = 4) Past Covariates
gbmD_4_past_covs = concatenate(
    [
        btc_usd4_ts,
        price_ts.slice_intersect(btc_usd4_ts),
        vol_ts.slice_intersect(btc_usd4_ts),
        lda_price.slice_intersect(btc_usd4_ts),
    ],
    axis=1,
)

In [31]:
gbmD_4_best_params, gbmD_4_min_error = gridsearch_GBM(
    btc_usd4_ts, gbmD_4_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:13<00:00, 34.61s/it]

Average error (min_error) over all historical forecasts: 1186.64
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 12

In [32]:
# Model D (h = 12) Past Covariates
gbmD_12_past_covs = concatenate(
    [
        btc_usd12_ts,
        price_ts.slice_intersect(btc_usd12_ts),
        vol_ts.slice_intersect(btc_usd12_ts),
        lda_price.slice_intersect(btc_usd12_ts),
    ],
    axis=1,
)

In [33]:
gbmD_12_best_params, gbmD_12_min_error = gridsearch_GBM(
    btc_usd12_ts, gbmD_12_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:01<00:00, 33.83s/it]

Average error (min_error) over all historical forecasts: 78.04
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### Model E (Price Returns, Price & Volume + Top2Vec Reddit Price Index)

#### h = 1

In [34]:
# Model E (h = 1) Past Covariates
gbmE_1_past_covs = concatenate(
    [
        btc_usd1_ts,
        price_ts.slice_intersect(btc_usd1_ts),
        vol_ts.slice_intersect(btc_usd1_ts),
        t2v_price.slice_intersect(btc_usd1_ts),
    ],
    axis=1,
)

In [35]:
gbmE_1_best_params, gbmE_1_min_error = gridsearch_GBM(
    btc_usd1_ts, gbmE_1_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:25<00:00, 35.36s/it]

Average error (min_error) over all historical forecasts: 258.84
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 4

In [36]:
# Model E (h = 4) Past Covariates
gbmE_4_past_covs = concatenate(
    [
        btc_usd4_ts,
        price_ts.slice_intersect(btc_usd4_ts),
        vol_ts.slice_intersect(btc_usd4_ts),
        t2v_price.slice_intersect(btc_usd4_ts),
    ],
    axis=1,
)

In [37]:
gbmE_4_best_params, gbmE_4_min_error = gridsearch_GBM(
    btc_usd4_ts, gbmE_4_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:33<00:00, 35.82s/it]

Average error (min_error) over all historical forecasts: 1140.39
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 12

In [38]:
# Model E (h = 12) Past Covariates
gbmE_12_past_covs = concatenate(
    [
        btc_usd12_ts,
        price_ts.slice_intersect(btc_usd12_ts),
        vol_ts.slice_intersect(btc_usd12_ts),
        t2v_price.slice_intersect(btc_usd12_ts),
    ],
    axis=1,
)

In [39]:
gbmE_12_best_params, gbmE_12_min_error = gridsearch_GBM(
    btc_usd12_ts, gbmE_12_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:43<00:00, 36.47s/it]

Average error (min_error) over all historical forecasts: 85.94
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### Model F (Price Returns, Price & Volume + VCRIX)

#### h = 1

In [41]:
# Model F (h = 1) Past Covariates
gbmF_1_past_covs = concatenate(
    [
        btc_usd1_ts,
        price_ts.slice_intersect(btc_usd1_ts),
        vol_ts.slice_intersect(btc_usd1_ts),
        t2v_price.slice_intersect(btc_usd1_ts),
    ],
    axis=1,
)

In [42]:
gbmF_1_best_params, gbmF_1_min_error = gridsearch_GBM(
    btc_usd1_ts, gbmF_1_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [17:51<00:00, 66.99s/it]

Average error (min_error) over all historical forecasts: 258.84
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 4

In [43]:
# Model F (h = 4) Past Covariates
gbmF_4_past_covs = concatenate(
    [
        btc_usd4_ts,
        price_ts.slice_intersect(btc_usd4_ts),
        vol_ts.slice_intersect(btc_usd4_ts),
        t2v_price.slice_intersect(btc_usd4_ts),
    ],
    axis=1,
)

In [44]:
gbmF_4_best_params, gbmF_4_min_error = gridsearch_GBM(
    btc_usd4_ts, gbmF_4_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [09:11<00:00, 34.46s/it]

Average error (min_error) over all historical forecasts: 1140.39
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}


#### h = 12

In [45]:
# Model F (h = 12) Past Covariates
gbmF_12_past_covs = concatenate(
    [
        btc_usd12_ts,
        price_ts.slice_intersect(btc_usd12_ts),
        vol_ts.slice_intersect(btc_usd12_ts),
        t2v_price.slice_intersect(btc_usd12_ts),
    ],
    axis=1,
)

In [46]:
gbmF_12_best_params, gbmF_12_min_error = gridsearch_GBM(
    btc_usd12_ts, gbmF_12_past_covs, 1, 1, 1
)

100%|██████████| 16/16 [08:37<00:00, 32.33s/it]

Average error (min_error) over all historical forecasts: 85.94
Best Params: {'boosting_type': 'gbdt', 'early_stopping_rounds': 30, 'feature_fraction': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'num_boost_round': 300, 'num_iterations': 500, 'num_leaves': 4, 'random_state': 42, 'subsample': 0.2}
